In [1]:
import os
from PIL import Image
import cv2
import numpy as np

In [2]:
use_folder = '../../datasets/data_vipr/masked_3/test/'
output_folder = '../../datasets/vegetation/test/'

In [3]:
def get_CIVE_band(img):
    '''
    Return the CIVE band calculated from an RGB image between 0 and 255
    using the formula below:
    CIVE = 0.441*red_channel - 0.881*green_channel + 0.385*blue_channel + 18.787
    '''
    img = cv2.GaussianBlur(img,(35,35),0)
    CIVE_band = 0.441*img[:,:,0] - 0.881*img[:,:,1] + 0.385*img[:,:,2] + 18.787
    normalized_CIVE_band = (((CIVE_band+abs(CIVE_band.min()))/CIVE_band.max())).astype(np.uint8)
    return normalized_CIVE_band

In [4]:
def apply_CIVE_mask(img, vegetation_index_band):
    '''
    Apply a binary mask on an image and return the masked image
    '''
    ret, otsu = cv2.threshold(vegetation_index_band,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    masked_img = cv2.bitwise_and(img,img,mask = otsu)
    return masked_img

In [5]:
os.makedirs(output_folder, exist_ok=True)

def apply_CIVE_mask_to_train_folder(input_folder, output_folder):
    for disease_folder in os.listdir(input_folder):
        disease_path = os.path.join(input_folder, disease_folder)

        # Check if it is a directory
        if not os.path.isdir(disease_path):
            continue

        # Create the corresponding disease subfolder in the output directory
        output_disease_folder = os.path.join(output_folder, disease_folder)
        os.makedirs(output_disease_folder, exist_ok=True)

        for image_name in os.listdir(disease_path):
            # Skip non-image files
            if not image_name.lower().endswith(('png', 'jpg', 'jpeg')):
                continue

            input_path = os.path.join(disease_path, image_name)
            output_path = os.path.join(output_disease_folder, image_name)

            # Load the image
            img = cv2.imread(input_path)

            # Check if the image is valid
            if img is None:
                print(f"Skipping invalid image: {input_path}")
                continue

            # Calculate the CIVE band
            CIVE_band = get_CIVE_band(img)

            # Apply the CIVE mask
            masked_img = apply_CIVE_mask(img, CIVE_band)

            # Save the masked image
            cv2.imwrite(output_path, masked_img)

            print(f"Processed and saved: {output_path}")

# Apply the CIVE mask to all images in the train folder
apply_CIVE_mask_to_train_folder(use_folder, output_folder)

print(f"All masked images have been saved to: {output_folder}")

Processed and saved: ../../datasets/vegetation/test/CBB\1003888281.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1012426959.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1027796931.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1036959902.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1072326766.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1094295442.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\109816879.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1127286868.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1185995427.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1203861431.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\121610069.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1222304315.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1236036550.jpg
Processed and saved: ../../datasets/vegetation/test/CBB\1241243048.jpg
Processe